<a href="https://colab.research.google.com/github/rajdas2001/News-Research-Tool-using-LLM/blob/main/News_Research_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!pip install ctransformers
!pip install chromadb
!pip install langchain
!pip install sentence_transformers
!pip install bitsandbytes
!pip install unstructured
!pip install transformers

In [49]:
# Importing libs

import pandas as pd
import numpy as np

import torch
from torch import bfloat16
import transformers
from transformers import AutoTokenizer

# LangChain
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.llms import CTransformers

from transformers import AutoModelForCausalLM

# Hiding warnings
import warnings
warnings.filterwarnings("ignore")

In [50]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [51]:
# Checking if GPU is available
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(torch.cuda.current_device())
    total_memory = torch.cuda.get_device_properties(0).total_memory
    total_memory_gb = total_memory / (1024**3) # Converting memory to Gb
    print("GPU is available. \nUsing GPU")
    print("\nGPU Name:", gpu_name)
    print(f"Total GPU Memory: {total_memory_gb:.2f} GB")

    device = torch.device('cuda')
else:
    print("GPU is not available. \nUsing CPU")
    device = torch.device('cpu')

GPU is not available. 
Using CPU


In [52]:
loader = UnstructuredURLLoader(urls=["https://www.moneycontrol.com/news/business/earnings/buy-hdfc-bank-target-of-rs-1850-icici-securities-12700761.html"])

In [53]:
document = loader.load()

In [54]:
# Splitting document into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                chunk_overlap = 20)
splitted_texts = splitter.split_documents(document)

In [55]:
# Loading model to create the embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [56]:
# Creating and indexed database
chroma_database = Chroma.from_documents(splitted_texts,
                                      embedding_model,
                                      persist_directory = 'chroma_db')

In [57]:
retriever = chroma_database.as_retriever()

In [58]:
# Configuring BitsAndBytesConfig for loading model in an optimal way
quantization_config = transformers.BitsAndBytesConfig(load_in_4bit = True,
                                        bnb_4bit_quant_type = 'nf4',
                                        bnb_4bit_use_double_quant = True,
                                        bnb_4bit_compute_dtype = bfloat16)

In [59]:
llm = CTransformers(model='TheBloke/Llama-2-7B-Chat-GGML',
                      model_type='llama',
                      config={'max_new_tokens': 600,
                              'temperature': 0.01,
                              'context_length': 2000}
                      )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [60]:
# Defining a QnA chain
QnA = RetrievalQA.from_chain_type(llm = llm,
                                 chain_type = 'stuff',
                                 retriever = retriever,
                                 verbose = False)

In [61]:
# Defining function to fetch documents according to a query
def get_answers(QnA, query):
    answer = QnA.run(query)
    print(f"\033[1mQuery:\033[0m {query}\n")
    print(f"\033[1mAnswer:\033[0m ", answer)

In [62]:
query = """What is the target price of HDFC?"""
get_answers(QnA, query)

Query: What is the target price of HDFC?

Answer:   The target price of HDFC according to ICICI Securities is Rs 1,850.
